In [1]:
import pandas as pd
import scipy.stats as stats 

In [2]:
df = pd.read_excel('../pcr_results/Aubrey update 21Mar23.xlsx')

### Discovered that beetle 51 has duplicate records.

<div class="alert-danger">
Decided not to correct this error.
</div>

In [3]:
df[df.duplicated(subset=['beetle_sn'], keep=False)]

,Date AgR received,Date CRB collected,Country,Site,Treatment,Sex,COI 1:100,COI RFLP (haplotype),Multiplex (OrNV),Virus,AgR batch label,Aubrey tube number,beetle_sn
101,2023-10-19,2023-07-20,Guam,"Leo Palace Golf Course, Guam",CONTROL,m,+,CRB-G,-,no,none,51,79
124,2023-11-03,2023-07-20,Guam,"Leo Palace Golf Course, Guam",CONTROL,m,faint,CRB-G,-,no,GUAM 202309,51,79


## Difference between proportion of OrNV positives in shipment 1 and 2 is highly significant 

||OrNV+|OrNV-|porportion positive|
|---|---|---|---|
|shipment 1|29   |88|25%|
|shipment 2|0    |54|0% |

p = 5.411e-06; Fisher's exact test

In [4]:
29/(29+88)

0.24786324786324787

In [5]:
df.groupby(['Date AgR received','Virus']).Virus.count()

Date AgR received  Virus
2023-10-19         faint     2
                   no       88
                   yes      27
2023-11-03         no       54
Name: Virus, dtype: int64

In [6]:
shipment1 = [29, 88]
shipment2 = [0, 54]
odd_ratio, p_value = stats.fisher_exact([shipment1, shipment2]) 
print('odd ratio is : ' + str(odd_ratio)) 
print('p_value is : ' + str(p_value)) 

odd ratio is : inf
p_value is : 5.411310925803131e-06


# Differences in proportions of OrNV positive beetles in shipment1

In [37]:
df_shipment1 = df[df['Date AgR received'] == '2023-10-19'].copy()
df_shipment1['Virus'].replace(to_replace='faint', value='yes', inplace=True)
xtab = pd.crosstab(df_shipment1.Treatment, df_shipment1.Virus)
xtab

Virus,no,yes
Treatment,,
CONTROL,34,1
DUG42,19,10
PNG,12,4
V23B,9,10
X2B,13,4


In [39]:
mydict = xtab.to_dict()
mydict

{'no': {'CONTROL': 34, 'DUG42': 19, 'PNG': 12, 'V23B': 9, 'X2B': 13},
 'yes': {'CONTROL': 1, 'DUG42': 10, 'PNG': 4, 'V23B': 10, 'X2B': 4}}

In [48]:
mylist = []
treatments = ['CONTROL','DUG42','PNG','V23B','X2B']
for treatment in treatments:
    mylist.append([mydict['no'][treatment], mydict['yes'][treatment]])
mylist                                                         

[[34, 1], [19, 10], [12, 4], [9, 10], [13, 4]]

In [49]:
import numpy as np
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
rpy2.robjects.numpy2ri.activate()

stats = importr('stats')
# m = np.array([[4,4],[4,5],[10,6]])
m = np.array(mylist)
res = stats.fisher_test(m)
print(f'p-value: {res[0][0]}')

p-value: 0.00028623328767403015


## Marascuillo post hoc test

In [51]:
import math
import itertools
import scipy.stats

degrees_of_freedom = 4
chi = math.sqrt(scipy.stats.chi2.ppf(0.975, degrees_of_freedom))
print(f'{chi=}')

pairs = list(itertools.combinations(treatments, 2))
mylist = []
for pair in pairs:
    g1 = pair[0]
    g2 = pair[1]
    diff = abs(df[df.group==g1].p_bad.values[0] - df[df.group==g2].p_bad.values[0])
    p1 = df[df.group==g1].p_bad.values[0]
    n1 = df[df.group==g1].n.values[0]
    p2 = df[df.group==g2].p_bad.values[0]
    n2 = df[df.group==g2].n.values[0]
    diff = abs(p1-p2)
    critical_range = chi * math.sqrt(((p1 * (1 - p1)) / n1) + ((p2 * (1 - p2)) / n2)) 
    if diff > critical_range:
        significant = 'yes'
    else:
        significant = 'no'                
    mylist.append({'g1': g1, 'g2': g2, 'diff': diff, 'critical_range': critical_range, 'significant': significant})
pd.DataFrame(mylist)

chi=3.338156194949211


AttributeError: 'DataFrame' object has no attribute 'group'